<h1><center>  Natural Language Processing - Análise de Letras de Música </center></h1> 

**Autora**: Natália da Silva Antunes <br>
**Contato**: nantunest@gmail.com <br>
**Portfólio**: [GitHub](github.com/natdsa)
_________________________________________________________


O objetivo deste estudo é empregar técnicas de Processamento de Linguagem Natural em letras de música de alguns artistas brasileiros. Foram coletadas todas as letras de músicas de cada artista, disponíveis no [Genius](https://genius.com). 

Nesse trabalho foram consideradas 360 letras de músicas dos seguintes artistas: Criolo, Fernandinho, Os Barões da Pisadinha e Pitty. A escolha dos artistas foi motivada por possuírem gêneros musicais distintos, possibilitando trazer uma maior diversidade ao conjunto de dados.

O trabalho foi desenvolvido utilizando a linguagem Python/ Jupyter Notebook. Todas as bibliotecas utilizadas encontram-se disponíveis neste. Para a representação textual foram utilizadas as seguintes técnicas de NLP: Embedding, similaridade de palavras, bigramas, Análise de Sentimentos (Textblob e Afinn).

Em resumo, o pipeline de dados foi realizado da seguinte maneira:
1.	Coleta de dados, mediante uso de API
2.	Agrupamento e tratamento dos dados
3.	Preprocessamento do texto
4.	Aplicação de técnicas de representação textual
5.	Aplicação de técnicas de análise de sentimentos
6.	Tópico por artista: Representação de palavras mais faladas e bigrama.



## 1. Criando o dataset

Para realizar a análise do conteúdo textual das músicas, será necessário a recuperação dessa informação. As letras de música serão extraidas do site [Genius](https://genius.com/), mediante uso da biblioteca lyricsgenius [Genius API Documentation](https://lyricsgenius.readthedocs.io/en/master/).
Caso não possua token, será necessário o registro na plataforma. [Acesse aqui](https://genius.com/api-clients/new).

Para este estudo, foram escolhidos previamente 4 artistas brasileiros, de gêneros musicais distintos. 



In [1]:
import pandas as pd
import os
import json
import re
from lyricsgenius import Genius
from os import listdir
from os.path import isfile, join
import numpy as np
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from unidecode import unidecode
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import word2vec
from textblob import TextBlob
from afinn import Afinn
from gensim.models import word2vec
import gensim
from tqdm._tqdm_notebook import tqdm_notebook
from nltk import bigrams
from afinn import Afinn

<ipython-input-1-7df6f2f6748d>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
## Set your token
mytoken='Set your token here'
genius = Genius(mytoken)

In [3]:
## Lista dos artistas
list_art=["Criolo", "Fernandinho","Pitty","Os Barões da Pisadinha"]

In [4]:
%%time
## Loop para recuperar as letras de todas as músicas de cada artista
for i in range(len(list_art)):
    artist = genius.search_artist(list_art[i], sort="title")
    artist.save_lyrics()

Searching for songs by Criolo...

Song 1: "4 da Manhã"
Song 2: "Agradeço a Quem Devo (Escuta)"
Song 3: "Ainda Há Tempo"
Song 4: "Andar como terrorista"
Song 5: "Aprendiz"
Song 6: "A Real"
Song 7: "Até Me Emocionei"
Song 8: "Boca de Lobo"
Song 9: "Boca Fofa"
Song 10: "Bogotá"
Song 11: "Breáco"
Song 12: "Calçada"
Song 13: "Cálice"
Song 14: "Capitulo 4, versiculo 3 - live"
Song 15: "Cartão de Visita"
Song 16: "Casa de Mãe (Acústico)"
Song 17: "Casa de Papelão"
Song 18: "Casca De Ovo"
Song 19: "Cerol"
Song 20: "Chuva Ácida"
Song 21: "Cigano"
Song 22: "Cóccix-ência"
Song 23: "Convoque Seu Buda"
Song 24: "Cria de Favela"
Song 25: "Demorô"
Song 26: "Demorô - live"
Song 27: "Dilúvio de Solidão"
Song 28: "Doum"
Song 29: "Duas de Cinco"
Song 30: "É o Teste"
Song 31: "Espiral de Ilusão"
Song 32: "Esquiva da Esgrima"
Song 33: "Etérea"
Song 34: "Fellini"
Song 35: "Fermento Pra Massa"
Song 36: "Filha do Maneco"
Song 37: "Fio De Prumo (Padê Onã)"
Song 38: "Freguês da Meia Noite"
Song 39: "Gelo No Inf

Song 1: "Agora Eu Pego Mesmo"
Song 2: "American Pie"
Song 3: "Amor da Despedida"
Song 4: "Amor Falso"
Song 5: "A Roça ou A Cidade"
Song 6: "Atrasadinha"
Song 7: "Baby Volta Pro Seu Nego"
Song 8: "Basta Você Me Ligar"
Song 9: "Batom de Ouro"
Song 10: "Bebe e Vem Me Procurar"
Song 11: "Bebezinha"
Song 12: "Beijo do Vampiro"
Song 13: "Cabeça Voando"
Song 14: "Casa de Praia"
Song 15: "Chama as Vaqueirinhas"
Song 16: "Chupadinha"
Song 17: "Com Ele é Cinema, Comigo é Cama"
Song 18: "Cornetinha"
Song 19: "Dançar Forró Beijando"
Song 20: "Desce do Cavalo"
Song 21: "Dim Dim Dim"
Song 22: "DNA do Interior"
Song 23: "Ela Não Larga Eu"
Song 24: "Escondido Dos Seus Pais"
Song 25: "Espinhaço do Véi"
Song 26: "Esquema Preferido"
Song 27: "Eu Acho Que Não"
Song 28: "Eu Fui Pra Terminar"
Song 29: "Fake News"
Song 30: "Fala Comigo Bebê (Ao Vivo)"
Song 31: "Festa na Roça"
Song 32: "Fiat Toro"
Song 33: "Filho do Mato"
Song 34: "Flecha Louca"
Song 35: "Fuleragem"
Song 36: "Galanteador"
Song 37: "Galera do 

In [7]:
## Carregando todos arquivos json que estão armazenados localmente
path_to_json = 'inserir aqui o caminho dos dados'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files)

## Setting novo dataframe
dados=[]
dados = pd.DataFrame(dados, columns=['artist','full_title','lyrics','lyrics_state'])

## Armazenando os dados em único dataframe
for i in range(len(json_files)):
    ## Open JSon file with lyrics saved
    f = open (json_files[i], "r")
    # Reading from file
    data = json.loads(f.read())
    df = pd.DataFrame(data['songs'])
    df=df[["artist","full_title","lyrics","lyrics_state"]]
    dados=dados.append(df)

['Lyrics_Criolo.json', 'Lyrics_Fernandinho.json', 'Lyrics_OsBarõesdaPisadinha.json', 'Lyrics_Pitty.json']


## 2.1 Pré processamento dos dados
Para a etapa de preprocessamento de dados foram utilizadas várias estratégias que foram apresendidas ao longo deste caminho de aprendizagem sobre NLP. Portanto, embora algumas estratégias sejam análogas, optou-se por mante-las nesse trabalho a fim de evidenciar as várias formas existentes de pre-prococessamento de textos. <br>


In [8]:
## Reset index
dados= dados.reset_index(drop=True)
dados

,artist,full_title,lyrics,lyrics_state
0,Criolo,4 da Manhã by Criolo,"Letra verificada de ""4 da Manhã"" com Criolo\n[...",complete
1,Criolo,Agradeço a Quem Devo (Escuta) by Criolo,"Ê, maior é Deus, pequeno sou eu\nO que eu tenh...",complete
2,Criolo,Ainda Há Tempo by Criolo,"[Verso 1]\nVocê quer saber, então vou te falar...",complete
3,Criolo,Andar como terrorista by Criolo,[Refrão]\nDesta vez o canhão não será necessár...,complete
4,Criolo,Aprendiz by Criolo,[Refrão 2x]\nEu quero algo de bom pra você e p...,complete
...,...,...,...,...
355,Pitty,Todos Estão Mudos by Pitty,"[Letra de ""Todos Estão Mudos"", de Pitty]\n\n[V...",complete
356,Pitty,Todos Estão Mudos - Ao Vivo by Pitty,"[Letra de ""Todos Estão Mudos - Ao Vivo"", de Pi...",complete
357,Pitty,Trapézio by Pitty,"[Letra de ""Trapézio"", de Pitty]\n\nEu queria e...",complete
358,Pitty,Trapézio - Ao Vivo by Pitty,"[Letra de ""Trapézio - Ao Vivo"", de Pitty]\n\nE...",complete


In [11]:
dados['lyrics'][2]

'[Verso 1]\nVocê quer saber, então vou te falar\nPorque as pessoas sadias adoecem\nBem alimentadas ou não, porque perecem?\nTudo está guardado na mente\nO que você quer nem sempre condiz com o que o outro sente\nEu tô falando é de atenção\nQue dá cola ao coração e faz marmanjo chorar\nSe faltar um simples sorriso\nOu às vezes um olhar\nE que se vem da pessoa errada não conta\nA amizade é importante, mas o amor escancara a tampa\nE o que te faz feliz também provoca a dor\nA cadência do surdo no coro que se forjou\nE aliás, cá pra nós, até o mais desandado\nDá um tempo na função quando percebe que é amado\nE as pessoas se olham e não se falam\nSe esbarram na rua e se maltratam\nUsam a desculpa de que nem Cristo agradou\nFalô, você vai querer mesmo se comparar com o Senhor?\n\n[Refrão]\nAs pessoas não são más\nElas só estão perdidas\nAinda há tempo\nNão quero ver\nVocê triste assim não\nQue a minha música possa\nTe levar amor\nNão quero ver\nVocê triste assim não\nQue a minha música possa

In [12]:
## Criando uma nova coluna para armazenar o texto que será limpo
## Substituir todos os textos dentro de parênteses/colchetes
dados['letra_limpa']=dados['lyrics']
for i in range(len(dados)):
    dados['letra_limpa'][i]=re.sub("[\(\[].*?[\)\]]", "", dados['letra_limpa'][i])

In [13]:
dados.head()

,artist,full_title,lyrics,lyrics_state,letra_limpa
0,Criolo,4 da Manhã by Criolo,"Letra verificada de ""4 da Manhã"" com Criolo\n[...",complete,"Letra verificada de ""4 da Manhã"" com Criolo\n\..."
1,Criolo,Agradeço a Quem Devo (Escuta) by Criolo,"Ê, maior é Deus, pequeno sou eu\nO que eu tenh...",complete,"Ê, maior é Deus, pequeno sou eu\nO que eu tenh..."
2,Criolo,Ainda Há Tempo by Criolo,"[Verso 1]\nVocê quer saber, então vou te falar...",complete,"\nVocê quer saber, então vou te falar\nPorque ..."
3,Criolo,Andar como terrorista by Criolo,[Refrão]\nDesta vez o canhão não será necessár...,complete,\nDesta vez o canhão não será necessário\nAnda...
4,Criolo,Aprendiz by Criolo,[Refrão 2x]\nEu quero algo de bom pra você e p...,complete,\nEu quero algo de bom pra você e pra mim\nE p...


In [14]:
## Métodos de preprocessamento de texto, porém com saídas distintas
def pre_processamento_texto(corpus):
    corpus_alt=re.findall(r"\w+(?:'\w+)?|[^\w\s]",corpus)
    #Lowcase
    corpus_alt=[t.lower() for t in corpus_alt]
    #remove stopwords"
    portugues_stops=stopwords.words('portuguese')
    corpus_alt=[t for t in corpus_alt if t not in portugues_stops]
    #remove números
    corpus_alt=[re.sub(r'\d','',t) for t in corpus_alt]
    #remove pontuação
    corpus_alt=[t for t in corpus_alt if t not in string.punctuation]
    #remove acentos
    corpus_alt=[unidecode(t) for t in corpus_alt]
    
    return corpus_alt

def pre_processamento_texto_return_str(corpus):
    #print("Documento")
    #print("#tokenização")
    corpus_alt=re.findall(r"\w+(?:'\w+)?|[^\w\s]",corpus)
    #Lowcase
    corpus_alt=[t.lower() for t in corpus_alt]
    #print("#remove stopwords")
    portugues_stops=stopwords.words('portuguese')
    corpus_alt=[t for t in corpus_alt if t not in portugues_stops]
    #print("#remove pontuação")
    corpus_alt=[t for t in corpus_alt if t not in string.punctuation]
   # print("#remove numeros")
    corpus_alt=[re.sub(r'\d','',t) for t in corpus_alt]

    corpus_alt_str=' '.join(corpus_alt)

      
    return corpus_alt_str

## 2.2 Representação Textual - Bigramas
Após a utilização de um método para realização do preprocessamento dos dados, sereá realizada a representação textual das letras de músicas por meio do cálculo dos bigramas.

In [16]:
letras_list = dados['letra_limpa'].values.tolist()
documento_limpo= [pre_processamento_texto(doc)  for doc in letras_list]
model_corpus_phrases = gensim.models.Phrases(documento_limpo, min_count=2)
#calulando os bigrams do corpus processado
bigram_corpus = model_corpus_phrases[documento_limpo]
bigram_corpus[2]

['quer_saber',
 'entao',
 'vou',
 'falar',
 'porque',
 'pessoas',
 'sadias',
 'adoecem',
 'bem',
 'alimentadas',
 'porque',
 'perecem',
 'tudo',
 'guardado',
 'mente',
 'quer',
 'sempre',
 'condiz',
 'outro',
 'sente',
 'to_falando',
 'atencao',
 'da',
 'cola',
 'coracao',
 'faz',
 'marmanjo',
 'chorar',
 'faltar',
 'simples',
 'sorriso',
 'vezes',
 'olhar',
 'vem',
 'pessoa',
 'errada',
 'conta',
 'amizade',
 'importante',
 'amor',
 'escancara',
 'tampa',
 'faz',
 'feliz',
 'provoca',
 'dor',
 'cadencia',
 'surdo',
 'coro',
 'forjou',
 'alias',
 'ca',
 'pra',
 'desandado',
 'da',
 'tempo',
 'funcao',
 'percebe',
 'amado',
 'pessoas',
 'olham',
 'falam',
 'esbarram',
 'rua',
 'maltratam',
 'usam',
 'desculpa',
 'cristo',
 'agradou',
 'falo',
 'vai',
 'querer',
 'comparar',
 'senhor',
 'pessoas_mas',
 'perdidas_ainda',
 'tempo',
 'quero_ver',
 'triste_assim',
 'musica_possa',
 'levar_amor',
 'quero_ver',
 'triste_assim',
 'musica_possa',
 'levar_amor',
 'exemplo',
 'to',
 'longe',
 'ser

## 2.3  Bag of Words

In [19]:
tqdm_notebook.pandas()
dados["text_sem_stopwords"]=dados['letra_limpa'].progress_apply(lambda x: pre_processamento_texto_return_str(x))
vec_bag=CountVectorizer()
X_bag=vec_bag.fit_transform(dados['text_sem_stopwords'])

  0%|          | 0/360 [00:00<?, ?it/s]

In [20]:
dados['text_sem_stopwords']

0      letra verificada  manhã criolo quatro manhã ac...
1      ê maior deus pequeno deus deu dou deus deu ê m...
2      quer saber então vou falar porque pessoas sadi...
3      desta vez canhão necessário andar terrorista f...
4      quero algo bom pra pra mim pra outra pessoa po...
                             ...                        
355    ouço clamores sinal frases outrora gritos supr...
356    ouço clamores sinal frases outrora gritos supr...
357    queria dizer diferente todo mundo sente conseg...
358    queria dizer diferente todo mundo sente conseg...
359    ei bem faz favor traz alguém saiba amor porém ...
Name: text_sem_stopwords, Length: 360, dtype: object

In [63]:
X_bag

<360x6432 sparse matrix of type '<class 'numpy.int64'>'
	with 20362 stored elements in Compressed Sparse Row format>

## 2.4 Embedding
Utilizando um embedding treinado

In [21]:
path = "inserir o caminho aqui"
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(path, binary=False)

In [22]:
#def get_embedding(frase):
#  return np.mean(np.array([word_vectors[palavra] for palavra in frase if palavra in word_vectors.key_to_index ]),axis=0)

In [23]:
#dados["text_sem_token"]=dados["letra_limpa"].progress_apply(lambda x: pre_processamento_texto(x))
#x_emb=dados["text_sem_token"].progress_apply(lambda x: get_embedding(x))

  0%|          | 0/360 [00:00<?, ?it/s]

  0%|          | 0/360 [00:00<?, ?it/s]

Gerando os embeddings com o texto processado das letras de música

In [25]:
model = word2vec.Word2Vec(dados["text_sem_token"], min_count=10, workers=4, seed=123, sg=1,  window=5)

In [26]:
##Similaridade entre palavras
model.wv.similarity('teto','vidro') ## Duas palavras presentes na música "Teto de Vidro" da cantora Pitty

0.99743986

In [27]:
def get_similares(model, palavra, max=5):
    palavras_similares=model.wv.most_similar(palavra, topn=5)
    palavras_similares=[p[0] for p in palavras_similares]
    palavras_similares.append(palavra)
    return palavras_similares

In [28]:
## Palavras similares de acordo com o modelo treinado
get_similares(model, "saudade")

['querendo', 'liga', 'senao', 'vontade', 'zero', 'saudade']

In [35]:
## Importante!! Só é possível retornar palavras similares de acordo com as palavras utilizadas no treinamento do modelo
origem=[ "amor", "dor", "teto","deus","saber","sofrer"]

for p in origem:
    print(get_similares(model, p))
    print()

['despedida', 'pes', 'vida', 'fala', 'cabeca', 'amor']

['feito', 'deixou', 'si', 'filhos', 'importa', 'dor']

['vidro', 'atire', 'primeira', 'pedra', 'tantas', 'teto']

['perdido', 'compara', 'vir', 'melhor', 'ouvirei', 'deus']

['deixar', 'acha', 'contar', 'sinto', 'ce', 'saber']

['morrer', 'tanto', 'traz', 'bola', 'biqueira', 'sofrer']



## 3 Análise de Sentimentos
Para essa etapa, serão utilizados dois tipos de algoritmos: <br>
* Textblob
* Afinn

In [37]:
result=[]
for i in range(len(dados)):
    sentence=TextBlob(dados.loc[i,"text_sem_stopwords"])
    result.append(sentence.sentiment)
dados['TextBlob_score']=result

In [41]:
afinn = Afinn()
result=[]
for i in range(len(dados)):
    result.append(afinn.score( dados.loc[i,"text_sem_stopwords"]))
dados['afinn_score']=result

In [43]:
## Retornar as letras/artista com Affin score menor do que zero
dados.loc[dados.afinn_score < 0,]

,artist,full_title,lyrics,lyrics_state,letra_limpa,text_sem_stopwords,text_sem_token,TextBlob_score,afinn_score
3,Criolo,Andar como terrorista by Criolo,[Refrão]\nDesta vez o canhão não será necessár...,complete,\nDesta vez o canhão não será necessário\nAnda...,desta vez canhão necessário andar terrorista f...,"[desta, vez, canhao, necessario, andar, terror...","(0.15833333333333333, 0.4666666666666666)",-10.0
10,Criolo,Breáco by Criolo,[Refrão x2]\nSó pode falar de vida quem vive\n...,complete,\nSó pode falar de vida quem vive\nSó pode fal...,pode falar vida vive pode falar sofrimento sof...,"[pode, falar, vida, vive, pode, falar, sofrime...","(-0.4, 0.4)",-2.0
24,Criolo,Demorô by Criolo,"[Intro]\nChorando\nNão, Criolo de novo não, nã...",complete,"\nChorando\nNão, Criolo de novo não, não quero...",chorando criolo novo quero então rá hô demorô ...,"[chorando, criolo, novo, quero, entao, ra, ho,...","(0.5, 0.5)",-2.0
27,Criolo,Doum by Criolo,Coloquei a melhor roupa\nE as crianças mandei ...,complete,Coloquei a melhor roupa\nE as crianças mandei ...,coloquei melhor roupa crianças mandei chamar t...,"[coloquei, melhor, roupa, criancas, mandei, ch...","(0.0, 0.0)",-1.0
29,Criolo,É o Teste by Criolo,[Verso 1]\nA todo momento provar que sou tranq...,complete,\nA todo momento provar que sou tranquilo\nNão...,todo momento provar tranquilo ando cano pra co...,"[todo, momento, provar, tranquilo, ando, cano,...","(0.5, 0.5)",-5.0
36,Criolo,Fio De Prumo (Padê Onã) by Criolo (Ft. Juçara ...,[Refrão: Juçara Marçal]\nLaroyê Bará\nAbra o c...,complete,\nLaroyê Bará\nAbra o caminho dos passos\nAbra...,laroyê bará abra caminho passos abra caminho o...,"[laroye, bara, abra, caminho, passos, abra, ca...","(0.2, 0.30000000000000004)",-3.0
60,Criolo,Rap é Forte by Criolo (Ft. Terra Preta),[Intro: Criolo]\nDo consumidor de uísque ao be...,complete,\nDo consumidor de uísque ao bebedor de cajuín...,consumidor uísque bebedor cajuína tamo junto c...,"[consumidor, uisque, bebedor, cajuina, tamo, j...","(-0.6, 0.7666666666666666)",-11.0
66,Criolo,Sucrilhos by Criolo,"[Verso 1]\nCalçada pra favela, avenida pra car...",complete,"\nCalçada pra favela, avenida pra carro\nCéu p...",calçada pra favela avenida pra carro céu pra a...,"[calcada, pra, favela, avenida, pra, carro, ce...","(0.03333333333333333, 0.06666666666666667)",-2.0
160,Os Barões da Pisadinha,Fake News by Os Barões da Pisadinha,Da minha boca só queria um beijo\nUsar o meu c...,complete,Da minha boca só queria um beijo\nUsar o meu c...,boca queria beijo usar corpo pra matar desejo ...,"[boca, queria, beijo, usar, corpo, pra, matar,...","(-0.5, 1.0)",-12.0
201,Os Barões da Pisadinha,Seu Bebê Tá Bebo by Os Barões da Pisadinha,"[Letra de ""Seu Bebê Tá Bebo"" com Os Barões da ...",complete,"\n\n\nApaixonadinho\nSó casa, cinema, sorvete,...",apaixonadinho casa cinema sorvete sushi açaí a...,"[apaixonadinho, casa, cinema, sorvete, sushi, ...","(0.0, 0.0)",-12.0


In [61]:
## Menor score
print(dados['lyrics'][270])

[Letra de "Ignorin’u - Ao Vivo", de Pitty]

I am ignoring you
I will ignore you
I am ignoring you
I will ignore you
Cause your heart is so
Full of this shit
And I will ignore you

Don't bring me your sorrows
Don't bring me your jealousy
Don't drown me in your mood
That kind of thing you do
Cause I was not born to be
The princess up the tower
Oh, you can't keep me
Quiet and tied in this bloody cage

I am ignoring you
I will ignore you
I am ignoring you
I will ignore you
Cause your heart is so
Full of this shit
And I will ignore you, you

I am not waiting for
A prince on a white horse just to save me
I know I have to do by myself
And this time I will be free

I am ignoring you
I will ignore you
I am ignoring you
I will ignore you
Cause your heart is so
Full of this shit
And I will ignore
I will ignore
I will ignore you
Ignoring you
Ignoring you


In [44]:
## Retornar as letras/artistas com Affin score maior do que zero
dados.loc[dados.afinn_score > 0,]

,artist,full_title,lyrics,lyrics_state,letra_limpa,text_sem_stopwords,text_sem_token,TextBlob_score,afinn_score
2,Criolo,Ainda Há Tempo by Criolo,"[Verso 1]\nVocê quer saber, então vou te falar...",complete,"\nVocê quer saber, então vou te falar\nPorque ...",quer saber então vou falar porque pessoas sadi...,"[quer, saber, entao, vou, falar, porque, pesso...","(0.0, 0.0)",1.0
4,Criolo,Aprendiz by Criolo,[Refrão 2x]\nEu quero algo de bom pra você e p...,complete,\nEu quero algo de bom pra você e pra mim\nE p...,quero algo bom pra pra mim pra outra pessoa po...,"[quero, algo, bom, pra, pra, mim, pra, outra, ...","(0.2, 0.2)",1.0
14,Criolo,Cartão de Visita by Criolo (Ft. Tulipa Ruiz),[Verso 1: Criolo]\nAcende o incenso de mirra f...,complete,\nAcende o incenso de mirra francesa\nAlgodão ...,acende incenso mirra francesa algodão fio toa...,"[acende, incenso, mirra, francesa, algodao, fi...","(-0.8, 1.0)",2.0
20,Criolo,Cigano by Criolo (Ft. The Abyssinians),"[Letra de ""Cigano"" com Criolo & The Abyssinian...",complete,\n\n\nJah-Van\nJah-Van\nJah-Van\nJah-Van\n\n\n...,jah van jah van jah van jah van querer viver p...,"[jah, van, jah, van, jah, van, jah, van, quere...","(0.0, 0.0)",3.0
21,Criolo,Cóccix-ência by Criolo,[Intro]\nDeixa o sangue coalhar\nDeixa o orval...,complete,\nDeixa o sangue coalhar\nDeixa o orvalho seca...,deixa sangue coalhar deixa orvalho secar amanh...,"[deixa, sangue, coalhar, deixa, orvalho, secar...","(0.0, 0.0)",6.0
28,Criolo,Duas de Cinco by Criolo,[Refrão]\nCompro uma pistola do vapor\nVisto o...,complete,\nCompro uma pistola do vapor\nVisto o jaco Ca...,compro pistola vapor visto jaco califórnia azu...,"[compro, pistola, vapor, visto, jaco, californ...","(0.0, 0.25)",36.0
30,Criolo,Espiral de Ilusão by Criolo,[Refrão]\nComo você dorme com isso?\nComo você...,complete,\nComo você dorme com isso?\nComo você dorme t...,dorme dorme tranquilo magoou destruiu desandou...,"[dorme, dorme, tranquilo, magoou, destruiu, de...","(0.0, 0.75)",2.0
31,Criolo,Esquiva da Esgrima by Criolo,[Verso 1]\nFalar demais chiclete azeda\nChama ...,complete,\nFalar demais chiclete azeda\nChama o SAMU e ...,falar demais chiclete azeda chama samu ensina ...,"[falar, demais, chiclete, azeda, chama, samu, ...","(-0.20833333333333331, 0.5916666666666667)",1.0
57,Criolo,Plano de Voo by Criolo (Ft. Síntese),[Verso 1: Criolo]\nE por mais que eu tente exp...,complete,"\nE por mais que eu tente explicar, não consig...",tente explicar consigo tornar concreto abstrat...,"[tente, explicar, consigo, tornar, concreto, a...","(0.1, 0.4)",2.0
59,Criolo,Que Bloco É Esse? by Criolo (Ft. Ilê Aiyê),[Verso 1: Criolo]\nHoje terra vai tremer\nHoje...,complete,\nHoje terra vai tremer\nHoje terra vai tremer...,hoje terra vai tremer hoje terra vai tremer vu...,"[hoje, terra, vai, tremer, hoje, terra, vai, t...","(0.016666666666666677, 0.31666666666666665)",2.0


In [62]:
## Maior score

print(dados['lyrics'][28])

[Refrão]
Compro uma pistola do vapor
Visto o jaco Califórnia azul
Faço uma mandinga pro terror
E vou...

[Verso 1]
É o cão, é o cânhamo, é o desamor
É o canhão na boca de quem tanto se humilhou
Inveja é um desgraça, alastra ódio e rancor
E cocaína é uma igreja gringa de le chereau
Pra cada rap escrito é uma alma que se salva
O rosto do carvoeiro é o Brasil que mostra a cara
Muito blá se fala, e a língua é uma piranha
Aqui é só trabalho, sorte é pras crianças
Que vê o professor em desespero na miséria
Que no meio do caminho da educação havia uma pedra
E havia uma pedra no meio do caminho
Ele não é preto véi mas no bolso leva um cachimbo
É o sleazestack, dos zóio branco, repara o brilho
Chewbacca na penha é maizena com pó de vidro
Comerciais de TV, glamour pra alcoolismo
É o Kinect do XBox por duas buchas de cinco
HA-HA-HA-HA-HA-HA
HA-HA-HA-HA-HA-HA
Chega a rir de nervoso, comédia vai chorar

[Refrão]
Compro uma pistola do vapor
Visto o jaco Califórnia azul
Faço uma mandinga pro terror
E

## 4 Tópicos - Análises Por Artistas
Realização de outras análises selecionando apenas o artista

## 4.1 Criolo - Rapper

In [45]:
criolo=dados.loc[dados['artist']=='Criolo']

In [46]:
## Ocorrencia palavras mais ditas
words = criolo['text_sem_token'].dropna()\
                            #.apply(lambda y: pd.value_counts(re.findall('([\s]\w+[\s])',' '.join(y))))\
                            .sum(axis=0)\
                            .to_frame()\
                            .reset_index()\
                            .sort_values(by=0,ascending=False)
words.columns = ['word','occurences']

In [47]:
words.head()

,word,occurences
0,pra,184.0
63,vai,69.0
127,vou,50.0
274,pa,46.0
186,ta,42.0


In [48]:
bigramseries = pd.Series([word for sublist in criolo['text_sem_token'].dropna()\
                    .apply(lambda x: [i for i in bigrams(x)])\
                    .tolist() for word in sublist])\
                    .value_counts()

In [51]:
bigramseries.head(20)

(pa, pa)             78
(homo, homo)         36
(oh, laraie)         32
(luta, luta)         32
(laraie, oh)         28
(ninguem, vai)       28
(pra, ver)           27
(ha, ha)             24
(e, mario)           24
(adjo, e)            24
(ogum, adjo)         24
(pra, mim)           22
(mario, ogum)        22
(to, pra)            19
(ver, daqui)         19
(aguas, mar)         18
(daqui, sucumbir)    17
(ai, ai)             16
(quero, ver)         13
(vai, chorar)        13
dtype: int64

## 4.2 Fernandinho - Artista Gospel

In [55]:
fernandinho=dados.loc[dados['artist']=='Fernandinho']
## Ocorrencia palavras mais ditas
words = fernandinho['text_sem_token'].dropna()\
                            #.apply(lambda y: pd.value_counts(re.findall('([\s]\w+[\s])',' '.join(y))))\
                            .sum(axis=0)\
                            .to_frame()\
                            .reset_index()\
                            .sort_values(by=0,ascending=False)
words.columns = ['word','occurences']
words.head()

,word,occurences
32,deus,66.0
93,mim,62.0
92,santo,52.0
195,jesus,45.0
43,ti,39.0


In [56]:
bigramseries = pd.Series([word for sublist in fernandinho['text_sem_token'].dropna()\
                    .apply(lambda x: [i for i in bigrams(x)])\
                    .tolist() for word in sublist])\
                    .value_counts()
bigramseries.head(20)

(santo, santo)       60
(pra, mim)           56
(oh, oh)             35
(deus, melhor)       28
(melhor, pra)        28
(jesus, jesus)       25
(chuva, chuva)       24
(cantam, santo)      22
(galileu, jesus)     22
(nova, historia)     20
(deus, pra)          20
(leva, leva)         18
(todos, cantam)      18
(jesus, galileu)     17
(compara, vir)       16
(nunca, atingido)    16
(santo, todos)       15
(mim, perdido)       14
(rei, reis)          14
(mim, deus)          14
dtype: int64

## 4.3 Os Barões da Pisadinha - Forró

In [57]:
baroes=dados.loc[dados['artist']=='Os Barões da Pisadinha']
## Ocorrencia palavras mais ditas
words = baroes['text_sem_token'].dropna()\
                            #.apply(lambda y: pd.value_counts(re.findall('([\s]\w+[\s])',' '.join(y))))\
                            .sum(axis=0)\
                            .to_frame()\
                            .reset_index()\
                            .sort_values(by=0,ascending=False)
words.columns = ['word','occurences']
words.head()

,word,occurences
3,pra,124.0
31,amor,80.0
20,ta,64.0
32,vou,56.0
18,vai,52.0


In [58]:
bigramseries = pd.Series([word for sublist in baroes['text_sem_token'].dropna()\
                    .apply(lambda x: [i for i in bigrams(x)])\
                    .tolist() for word in sublist])\
                    .value_counts()
bigramseries.head(20)

(oh, oh)                48
(e, e)                  20
(din, din)              16
(fazer, amor)           15
(vai, ter)              14
(espinhaco, veio)       14
(ah, ah)                14
(pra, mim)              13
(voltar, pra)           13
(amor, despedida)       13
(to, zero)              12
(bebo, amor)            12
(esquema, preferido)    12
(senta, aqui)           12
(ei, ei)                12
(veio, quebrou)         12
(bebe, ta)              12
(gente, faz)            12
(aqui, senta)           12
(vai, doer)             12
dtype: int64

## 4.4 Pitty - Rock

In [59]:
pitty=dados.loc[dados['artist']=='Pitty']
## Ocorrencia palavras mais ditas
words = baroes['text_sem_token'].dropna()\
                           #.apply(lambda y: pd.value_counts(re.findall('([\s]\w+[\s])',' '.join(y))))\
                            .sum(axis=0)\
                            .to_frame()\
                            .reset_index()\
                            .sort_values(by=0,ascending=False)
words.columns = ['word','occurences']
words.head()

,word,occurences
3,pra,124.0
31,amor,80.0
20,ta,64.0
32,vou,56.0
18,vai,52.0


In [60]:
bigramseries = pd.Series([word for sublist in pitty['text_sem_token'].dropna()\
                    .apply(lambda x: [i for i in bigrams(x)])\
                    .tolist() for word in sublist])\
                    .value_counts()
bigramseries.head(20)

(roda, gira)          44
(gira, gira)          39
(gira, roda)          39
(roda, roda)          34
(ah, ah)              33
(homem, lobo)         32
(bizarro, bizarro)    30
(semana, vem)         28
(oh, oh)              27
(namah, shivaya)      26
(om, namah)           26
(lobo, homem)         26
(queimar, novo)       25
(tarde, demais)       25
(nunca, tarde)        25
(faz, queimar)        25
(gente, junta)        24
(junta, pra)          24
(onde, ir)            24
(i, will)             24
dtype: int64

## 5 Conclusão

As letras de música são uma excelente forma de estudar representação textual.O conteúdo é bastante rico, tanto pela poética - vide Criolo - ou expressões populares, como as canções de Os Barões da Pisadinha. <br>

Como ponto positivo, destaca-se o metodo de encontrar similaridade entre as palavras!! 

Um ponto de dificuldade encontrado está nas representações como "ah", "ha", que normalmente fazem parte da fonética da canção. Além disso, outra dificuldade encontrada está na análise de sentimentos. Os métodos aqui estudados, na grande maioria das canções, obteve-se um score neutro.<br>

Para futuros trabalhos, sugere-se a utilização de canções por décadas. Pode-se levantar como hipótese se há diferença entre as palavras das músicas dos anos 80 para as palavras das canções dos anos 2010. Será que existe diferença?